In [ ]:
<div className={styles.chatMessages}>
  {chatMessages.map((message, index) => (
    <div
      key={index}
      className={`${styles.chatMessage} ${
        message.role === "user" 
          ? styles.userMessage 
          : message.role === "system" 
            ? styles.systemMessage 
            : styles.assistantMessage
      }`}
    >
      {message.role === "user" || message.role === "system" ? (
        message.content
      ) : (
        <div dangerouslySetInnerHTML={{ __html: message.content }} />
      )}
    </div>
  ))}
</div>

In [ ]:
const handleAction = async (userInput) => {
  setIsProcessing(true);
  setError(null);
  setStatusMessage(null);
  setProcessingSteps([]);
  
  // Immediately add the user's message to the chat
  setChatMessages(prevMessages => [...prevMessages, { role: "user", content: userInput }]);
  
  try {
    const action = userConfig.buttons[0];
    console.log(`Handling action: ${action.label}, User input: ${userInput}`);
    const content = await getEmailContent();
    const payload = {
      userId: userConfig.userId,
      emailContent: content,
      prompt: userInput,
    };

    console.log(`Sending request to: ${action.apiEndpoint}`);
    const response = await fetch(action.apiEndpoint, {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(payload),
    });

    if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
    
    const reader = response.body.getReader();
    const decoder = new TextDecoder();

    while (true) {
      const { value, done } = await reader.read();
      if (done) break;
      const decodedChunk = decoder.decode(value, { stream: true });
      const lines = decodedChunk.split('\n');
      
      for (const line of lines) {
        if (line.startsWith('data:')) {
          const data = JSON.parse(line.slice(5));
          if (data.type === 'step') {
            setProcessingSteps(prevSteps => [...prevSteps, data.content]);
          } else if (data.type === 'response') {
            const wrappedContent = wrapInHtml(data.content);
            setChatMessages(prevMessages => [
              ...prevMessages,
              { role: "assistant", content: wrappedContent }
            ]);
          }
        }
      }
    }
  } catch (e) {
    console.error(`Error in handleAction: ${e.message}`);
    setError(`Failed to process your request. Please try again.`);
  } finally {
    setIsProcessing(false);
    setProcessingSteps([]);
  }
};

In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
import json
import asyncio

app = FastAPI()

class EmailRequest(BaseModel):
    userId: str
    emailContent: str
    prompt: str

async def run_finance_agent_with_steps(prompt):
    # Simulate processing steps
    steps = [
        "Analyzing input",
        "Retrieving financial data",
        "Applying financial models",
        "Generating insights",
        "Formatting response"
    ]
    
    for step in steps:
        yield f"data: {json.dumps({'type': 'step', 'content': step})}\n\n"
        await asyncio.sleep(1)  # Simulate work
    
    # Simulate final response
    final_response = f"Here's the financial analysis for '{prompt}': ..."
    yield f"data: {json.dumps({'type': 'response', 'content': final_response})}\n\n"

@app.post("/generate_response")
async def generate_response(request: EmailRequest):
    async def event_generator():
        async for event in run_finance_agent_with_steps(request.prompt):
            yield event

    return StreamingResponse(event_generator(), media_type="text/event-stream")

@app.get("/getUserId")
async def get_user_id():
    return {"userId": "user1"}

@app.get("/getUserConfig/{user_id}")
async def get_user_config(user_id: str):
    # This is a mock configuration. In a real application, you'd fetch this from a database.
    return {
        "userId": user_id,
        "buttons": [
            {
                "label": "Financial Analysis",
                "icon": "CalculatorRegular",
                "apiEndpoint": "http://localhost:8000/generate_response",
                "description": "Generate a financial analysis based on the email content.",
            }
        ]
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
async def finance_agent_generator(query: str):
    graph = create_finance_agent_graph()
    initial_state = AgentState(
        query=query,
        processed_query="",
        use_graph_db=False,
        graph_db_result=[],
        expanded_queries=[],
        subgraphs_to_execute={},
        subgraph_results={},
        final_response=""
    )

    async def stream_updates():
        current_state = initial_state
        while True:
            new_state = await graph.ainvoke(current_state)
            if new_state == current_state:
                break
            
            # Stream the updates
            updates = {}
            if new_state.processed_query != current_state.processed_query:
                updates['processed_query'] = new_state.processed_query
            if new_state.graph_db_result != current_state.graph_db_result:
                updates['graph_db_result'] = new_state.graph_db_result
            if new_state.expanded_queries != current_state.expanded_queries:
                updates['expanded_queries'] = new_state.expanded_queries
            if new_state.subgraphs_to_execute != current_state.subgraphs_to_execute:
                updates['subgraphs_to_execute'] = list(new_state.subgraphs_to_execute.keys())
            if new_state.subgraph_results != current_state.subgraph_results:
                updates['subgraph_results'] = list(new_state.subgraph_results.keys())
            if new_state.final_response != current_state.final_response:
                updates['final_response'] = new_state.final_response
            
            if updates:
                yield f"data: {json.dumps(updates)}\n\n"
            
            current_state = new_state
            await asyncio.sleep(0.1)  # Small delay to avoid overwhelming the client

    return StreamingResponse(stream_updates(), media_type="text/event-stream"

In [ ]:
const handleAction = async (userInput) => {
  setIsProcessing(true);
  setError(null);
  setStatusMessage(null);
  setProcessingSteps([]);
  
  // Immediately add the user's message to the chat
  setChatMessages(prevMessages => [...prevMessages, { role: "user", content: userInput }]);
  
  try {
    const action = userConfig.buttons[0];
    console.log(`Handling action: ${action.label}, User input: ${userInput}`);
    const content = await getEmailContent();
    const payload = {
      userId: userConfig.userId,
      emailContent: content,
      prompt: userInput,
    };

    console.log(`Sending request to: ${action.apiEndpoint}`);
    const response = await fetch(action.apiEndpoint, {
      method: "POST",
      headers: { "Content-Type": "application/json" },
      body: JSON.stringify(payload),
    });

    if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
    
    const reader = response.body.getReader();
    const decoder = new TextDecoder();

    while (true) {
      const { value, done } = await reader.read();
      if (done) break;
      const chunk = decoder.decode(value);
      const lines = chunk.split('\n');
      
      for (const line of lines) {
        if (line.startsWith('data: ')) {
          const data = JSON.parse(line.slice(5));
          updateProcessingSteps(data);
        }
      }
    }
  } catch (e) {
    console.error(`Error in handleAction: ${e.message}`);
    setError(`Failed to process your request. Please try again.`);
  } finally {
    setIsProcessing(false);
  }
};

const updateProcessingSteps = (data) => {
  setProcessingSteps(prevSteps => {
    const newSteps = [...prevSteps];
    if (data.processed_query) newSteps.push(`Processed query: ${data.processed_query}`);
    if (data.graph_db_result) newSteps.push(`Database query completed`);
    if (data.expanded_queries) newSteps.push(`Expanded queries: ${data.expanded_queries.length}`);
    if (data.subgraphs_to_execute) newSteps.push(`Executing subgraphs: ${data.subgraphs_to_execute.join(', ')}`);
    if (data.subgraph_results) newSteps.push(`Subgraph results processed: ${data.subgraph_results.join(', ')}`);
    if (data.final_response) {
      newSteps.push(`Generating final response`);
      setChatMessages(prevMessages => [
        ...prevMessages,
        { role: "assistant", content: wrapInHtml(data.final_response) }
      ]);
    }
    return newSteps;
  });
};

In [ ]:
import React from 'react';
import { makeStyles, tokens } from "@fluentui/react-components";

const useStyles = makeStyles({
  progressContainer: {
    marginTop: '16px',
    padding: '16px',
    backgroundColor: tokens.colorNeutralBackground3,
    borderRadius: tokens.borderRadiusMedium,
  },
  progressStep: {
    marginBottom: '8px',
    padding: '8px',
    backgroundColor: tokens.colorNeutralBackground1,
    borderRadius: tokens.borderRadiusSmall,
    fontSize: tokens.fontSizeBase200,
    transition: 'opacity 0.3s ease-in-out',
  },
  completedStep: {
    opacity: 0.6,
  },
});

const DynamicProgressBar = ({ steps }) => {
  const styles = useStyles();

  return (
    <div className={styles.progressContainer}>
      {steps.map((step, index) => (
        <div 
          key={index} 
          className={`${styles.progressStep} ${index < steps.length - 1 ? styles.completedStep : ''}`}
        >
          {step}
        </div>
      ))}
    </div>
  );
};

export default DynamicProgressBar;

import React, { useState, useEffect, useCallback } from "react";
import { Button, Text, Spinner, makeStyles, shorthands, tokens } from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";
import DynamicProgressBar from "./DynamicProgressBar";

// ... (keep existing styles)

const EmailGenerator = () => {
  // ... (keep existing state and functions)

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatHeader}>
          <div className={styles.chatHeaderTitle}>
            <div className={styles.iconWrapper}>
              <FluentIcons.MailTemplate24Regular />
            </div>
            <Text className={styles.chatHeaderText}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text>
        </div>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <div
              key={index}
              className={`${styles.chatMessage} ${
                message.role === "user" ? styles.userMessage : styles.assistantMessage
              }`}
            >
              {message.role === "user" ? (
                message.content
              ) : (
                <div dangerouslySetInnerHTML={{ __html: message.content }} />
              )}
            </div>
          ))}
        </div>
        {isProcessing && <DynamicProgressBar steps={processingSteps} />}
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  // ... (keep the rest of the component as is)
};

export default EmailGenerator;